In [1]:
import numpy as np
import pandas as pd

In [2]:
pd.options.display.max_rows = 3000

In [3]:
df = pd.read_csv('./datasets/backup2.csv')
df.head()

,title,time,sub,comments
0,Dad reflexes prevent crash.,2 hours ago,r/DadReflexes,249 comments
1,Quite literally choosing beggar,4 hours ago,r/ChoosingBeggars,631 comments
2,When your cat crashes his bicycle in his dream...,4 hours ago,r/funny,642 comments
3,Heatmap of numbers found at the end of Reddit ...,5 hours ago,r/dataisbeautiful,2612 comments
4,Hre you OPEN TODAY,5 hours ago,r/oldpeoplefacebook,313 comments


In [4]:
df.shape

(2525, 4)

In [5]:
df['comments'] = df['comments'].apply(lambda x: x.replace('comments', ''))

In [6]:
df['comments'].value_counts()

2            121
1 comment     97
3             84
4             78
7             76
6             74
5             73
9             69
11            64
10            64
comment       63
14            59
8             57
12            49
15            42
18            38
13            37
20            31
22            30
16            29
23            28
17            28
21            25
37            25
31            24
28            23
19            23
24            22
26            21
35            20
27            20
25            16
29            16
32            15
30            15
42            14
36            14
34            14
64            13
38            13
39            13
43            12
45            11
33            11
40            11
57            10
68            10
50            10
65            10
41            10
79             9
61             9
60             9
77             9
47             9
80             9
53             9
51             8
59            

In [7]:
df['comments'] = df['comments'].apply(lambda x: x.replace('comment', ''))

In [8]:
df['comments'] = df['comments'].apply(lambda x: 0 if x == '' else x)

In [9]:
df['comments'] = df['comments'].apply(lambda x: int(x))

In [10]:
df.dtypes

title       object
time        object
sub         object
comments     int64
dtype: object

In [11]:
df['time'].value_counts()

6 hours ago       316
4 hours ago       309
5 hours ago       308
7 hours ago       279
3 hours ago       235
8 hours ago       199
2 hours ago       191
9 hours ago       117
10 hours ago       97
11 hours ago       74
1 hour ago         48
12 hours ago       43
13 hours ago       40
17 hours ago       36
15 hours ago       33
14 hours ago       28
16 hours ago       27
an hour ago        27
19 hours ago       24
18 hours ago       21
23 hours ago       19
20 hours ago       16
21 hours ago       12
22 hours ago       11
59 minutes ago      2
41 minutes ago      2
29 minutes ago      1
50 minutes ago      1
28 minutes ago      1
48 minutes ago      1
57 minutes ago      1
22 minutes ago      1
30 minutes ago      1
32 minutes ago      1
35 minutes ago      1
10 minutes ago      1
Name: time, dtype: int64

In [12]:
df['time'].apply(lambda x: type(x)).value_counts()

<type 'str'>      2524
<type 'float'>       1
Name: time, dtype: int64

In [13]:
df[df['time'].isnull() == True]

,title,time,sub,comments
1259,Turns out AB+ isn't always all that selfish,NaN,r/coolguides,174


In [14]:
df.isnull().sum()

title       0
time        1
sub         0
comments    0
dtype: int64

In [16]:
df['time'] = df['time'].apply(lambda x: float(str(x)[0:2])/60 if 'minutes' in str(x)\
                 else float(str(x)[:-10]) if 'hours' in str(x) else x)

In [17]:
df['time'] = df['time'].apply(lambda x: 1 if 'hour' in str(x) else x)

In [18]:
df.dtypes

title        object
time        float64
sub          object
comments      int64
dtype: object

In [22]:
df = df.dropna()

In [23]:
df.describe()

,time,comments
count,2524.000000,2524.000000
mean,6.845094,126.415214
std,4.405695,568.840644
min,0.166667,0.000000
25%,4.000000,7.000000
50%,6.000000,21.000000
75%,8.000000,73.000000
max,23.000000,17273.000000


In [28]:
len(df[df['comments'] >= 100])/float(len(df))

0.19730586370839936

In [29]:
# There is an extremely high variance in the number of comments. 
# Choosing 100 as a threshold seems reasonble as just under 20% of posts achieve that. 

In [32]:
df['H/L'] = df['comments'].apply(lambda x: 'Low' if x < 100 else 'High')

In [36]:
df.head()

,title,time,sub,comments,H/L
0,Dad reflexes prevent crash.,2.0,r/DadReflexes,249,High
1,Quite literally choosing beggar,4.0,r/ChoosingBeggars,631,High
2,When your cat crashes his bicycle in his dream...,4.0,r/funny,642,High
3,Heatmap of numbers found at the end of Reddit ...,5.0,r/dataisbeautiful,2612,High
4,Hre you OPEN TODAY,5.0,r/oldpeoplefacebook,313,High


In [37]:
df['H/L'].value_counts()

Low     2026
High     498
Name: H/L, dtype: int64

In [39]:
df['CpH'] = df['comments']/df['time']

In [48]:
df.describe()

,time,comments,CpH
count,2524.000000,2524.000000,2524.000000
mean,6.845094,126.415214,17.321115
std,4.405695,568.840644,62.392326
min,0.166667,0.000000,0.000000
25%,4.000000,7.000000,1.375000
50%,6.000000,21.000000,4.000000
75%,8.000000,73.000000,11.428571
max,23.000000,17273.000000,1107.500000


In [52]:
len(df[df['CpH'] >= 10])/float(len(df))

0.2777337559429477

In [53]:
# On second thought, maybe it's better to classify for a good rate - more than 10 comments per hour.
# About 28% of posts achieved that. 

In [55]:
df['H/L'] = df['CpH'].apply(lambda x: 'Low' if x < 10 else 'High')

In [56]:
df.head()

,title,time,sub,comments,H/L,CpH
0,Dad reflexes prevent crash.,2.0,r/DadReflexes,249,High,124.50
1,Quite literally choosing beggar,4.0,r/ChoosingBeggars,631,High,157.75
2,When your cat crashes his bicycle in his dream...,4.0,r/funny,642,High,160.50
3,Heatmap of numbers found at the end of Reddit ...,5.0,r/dataisbeautiful,2612,High,522.40
4,Hre you OPEN TODAY,5.0,r/oldpeoplefacebook,313,High,62.60


In [57]:
# Reordering the columns to put the target at the end. 
cols = ['title', 'time', 'sub', 'comments', 'CpH', 'H/L']
df = df[cols]

In [58]:
df.head()

,title,time,sub,comments,CpH,H/L
0,Dad reflexes prevent crash.,2.0,r/DadReflexes,249,124.50,High
1,Quite literally choosing beggar,4.0,r/ChoosingBeggars,631,157.75,High
2,When your cat crashes his bicycle in his dream...,4.0,r/funny,642,160.50,High
3,Heatmap of numbers found at the end of Reddit ...,5.0,r/dataisbeautiful,2612,522.40,High
4,Hre you OPEN TODAY,5.0,r/oldpeoplefacebook,313,62.60,High


In [59]:
df.iloc[3,0]

'Heatmap of numbers found at the end of Reddit usernames [OC]'